In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
kaggle_user=os.getenv('KAGGLE_USER')
kaggle_key=os.getenv('KAGGLE_KEY')
os.environ['KAGGLE_USERNAME'] = kaggle_user
os.environ['KAGGLE_KEY'] = kaggle_key

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [4]:
api.dataset_download_files('apoorvaappz/global-super-store-dataset', path=".")

In [34]:
import pandas as pd
import numpy as np

In [35]:
dataFrame=pd.read_csv("./global-super-store-dataset/Global_Superstore2.csv") 

In [48]:
dropingFields=["Row ID","Order ID","Order Date","Ship Date","Customer ID","Customer Name","Postal Code","Product ID"]
dataFrame.drop(dropingFields,axis=1,inplace=True)

In [49]:
dataFrame["Order Priority"].unique()

array([0, 1, 2, 3], dtype=int64)

# Changing Order priority to int 
### Critical - 0
### medium - 1
### High - 2
### Low - 3


In [50]:
dataFrame["Order Priority"].replace("Critical",0,inplace=True)
dataFrame["Order Priority"].replace("Medium",1,inplace=True)
dataFrame["Order Priority"].replace("High",2,inplace=True)
dataFrame["Order Priority"].replace("Low",3,inplace=True)
dataFrame

,Ship Mode,Segment,City,State,Country,Market,Region,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Item Value
0,Same Day,Consumer,New York City,New York,United States,US,East,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,0,329.950
1,Second Class,Corporate,Wollongong,New South Wales,Australia,APAC,Oceania,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,0,412.155
2,First Class,Consumer,Brisbane,Queensland,Australia,APAC,Oceania,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,1,575.019
3,First Class,Home Office,Berlin,Berlin,Germany,EU,Central,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,1,578.502
4,Same Day,Consumer,Dakar,Dakar,Senegal,Africa,Africa,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,0,354.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,Same Day,Corporate,Kure,Hiroshima,Japan,APAC,North Asia,"Advantus Thumb Tacks, 12 Pack",65.100,5,0.0,4.5000,0.01,1,13.020
51286,Standard Class,Consumer,Houston,Texas,United States,US,Central,Hoover Replacement Belt for Commercial Guardsm...,0.444,1,0.8,-1.1100,0.01,1,0.444
51287,Same Day,Home Office,Oxnard,California,United States,US,West,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.920,3,0.0,11.2308,0.01,2,7.640
51288,Standard Class,Home Office,Valinhos,São Paulo,Brazil,LATAM,South,"Acco Index Tab, Economy",13.440,2,0.0,2.4000,0.00,1,6.720


In [51]:
dataFrame["Item Value"]=dataFrame["Sales"]/dataFrame["Quantity"] 
list(dataFrame.iloc[0])

['Same Day',
 'Consumer',
 'New York City',
 'New York',
 'United States',
 'US',
 'East',
 'Plantronics CS510 - Over-the-Head monaural Wireless Headset System',
 2309.65,
 7,
 0.0,
 762.1845,
 933.57,
 0,
 329.95]

In [52]:
dataFrame["Product Name"].unique()

array(['Plantronics CS510 - Over-the-Head monaural Wireless Headset System',
       'Novimex Executive Leather Armchair, Black',
       'Nokia Smart Phone, with Caller ID', ...,
       'Kleencut Forged Office Shears by Acme United Corporation',
       'Holmes Visible Mist Ultrasonic Humidifier with 2.3-Gallon Output per Day, Replacement Filter',
       'Eureka Disposable Bags for Sanitaire Vibra Groomer I Upright Vac'],
      dtype=object)

In [53]:
x=dataFrame.drop("Order Priority",axis=1)
y=dataFrame["Order Priority"]
dataFrame.columns

Index(['Ship Mode', 'Segment', 'City', 'State', 'Country', 'Market', 'Region',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Shipping Cost', 'Order Priority', 'Item Value'],
      dtype='object')

In [54]:
x.iloc[85].values

array(['Second Class', 'Consumer', 'Bandung', 'Jawa Barat', 'Indonesia',
       'APAC', 'Southeast Asia', 'Hamilton Beach Refrigerator, Black',
       2487.8088, 6, 0.17, -269.7912, 562.14, 414.6348], dtype=object)

In [55]:
#Turn the categories into numbers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import json

categorical_features=["Ship Mode","Segment","City","State","Country","Market","Region","Product Name"]
# one_hot=OneHotEncoder()
# transformer=ColumnTransformer([(
#                                 "one_hot",
#                                 one_hot,
#                                 categorical_features
#                              )],
#                                 remainder="passthrough"
#                             )
# transformed_x = transformer.fit_transform(x)
# pd.DataFrame(transformed_x)
final_out_data={}
for i in categorical_features:
    unique_values=list(x[i].unique())
    for j in unique_values:
        if(i in final_out_data):
            final_out_data[i][j]=unique_values.index(j)
        else:
            final_out_data[i]={}
            final_out_data[i][j]=unique_values.index(j)
        x[i].replace(to_replace=j,value=int(unique_values.index(j)),inplace=True) 

import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(final_out_data, f, ensure_ascii=False, indent=4)
print("finished")

finished


In [88]:
np.random.seed(40)
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.2)

In [89]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((10258, 14), (41032, 14), (10258,), (41032,))

In [90]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)

In [91]:
clf.fit(x_train,y_train);

In [92]:
y_preds=clf.predict(x_test) 
y_preds

array([1, 2, 1, ..., 2, 1, 2], dtype=int64)

In [93]:
clf.score(x_test,y_test)

0.6824673425619029

In [94]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
accuracy_score(y_test,y_preds)

0.6824673425619029

In [95]:
import pickle
file = "finalModel.pkl";
fileObject=open(file,"wb");
pickle.dump(clf,fileObject);
fileObject.close();
fileObject2=open(file,"rb");
model=pickle.load(fileObject2,encoding='binary');
model.fit(x_train,np.ravel(y_train,order='C')); 
model.score(x_test,y_test)

0.6808344706570482